In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 500

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

In [6]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

In [7]:
# pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )

In [8]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )



In [9]:
len(pt_features[pt_features['isCase']==True])

15209

In [10]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,data_start,matchid,reason_for_removal,age_at_index_date,insomnia,stroke,heart_failure,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,chronic_pulmonary_disease,epilepsy,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,other_sedatives_100_pdds,antipsychotics_100_pdds,antidepressants_100_pdds,non_insomnia_GP_consultations,benzo_and_z_drugs_any,insomnia_any,insomnia_count:0,insomnia_count:1_5,insomnia_count:above_5,non_insomnia_GP_consultations:0,non_insomnia_GP_consultations:1_10,non_insomnia_GP_consultations:11_100,non_insomnia_GP_consultations:101_1000,non_insomnia_GP_consultations:above_1000,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,antidepressant_pdds:00000,antidepressant_pdds:00001_10,antidepressant_pdds:00011_100,antidepressant_pdds:00101_1000,antidepressant_pdds:01001_10000,antidepressant_pdds:10000_and_above,antipsychotic_pdds:00000,antipsychotic_pdds:00001_10,antipsychotic_pdds:00011_100,antipsychotic_pdds:00101_1000,antipsychotic_pdds:01001_10000,antipsychotic_pdds:10000_and_above,other_sedative_pdds:00000,other_sedative_pdds:00001_10,other_sedative_pdds:00011_100,other_sedative_pdds:00101_1000,other_sedative_pdds:01001_10000,other_sedative_pdds:10000_and_above,benzo_and_z_drug_pdds:00000,benzo_and_z_drug_pdds:00001_10,benzo_and_z_drug_pdds:00011_100,benzo_and_z_drug_pdds:00101_1000,benzo_and_z_drug_pdds:01001_10000,benzo_and_z_drug_pdds:10000_and_above,mood_stabiliser_pdds:00000,mood_stabiliser_pdds:00001_10,mood_stabiliser_pdds:00011_100,mood_stabiliser_pdds:00101_1000,mood_stabiliser_pdds:01001_10000,mood_stabiliser_pdds:10000_and_above,mental_illness
21423,2598600,39,600,1,2011-01-05,False,NaN,2013-01-25,1997-01-31,33218,NaN,72,0,0,0,1,0,0,0,0,0.0,0.000000,0.0,0.0,0.000000,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
28193,5856309,34,309,1,2004-09-23,True,1350.0,2013-04-16,1994-01-31,46620,NaN,70,0,0,0,1,0,0,0,0,0.0,0.000000,0.0,0.0,0.000000,50,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
28391,6491403,34,403,1,2011-04-01,True,8195.0,2013-05-10,1994-01-31,59709,NaN,77,0,0,0,0,0,0,0,0,0.0,0.000000,0.0,0.0,0.000000,200,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
28777,2200229,28,229,1,2010-04-16,True,1917.0,2011-10-31,1992-01-31,34976,NaN,82,1,0,0,0,0,0,0,0,0.0,0.159755,0.0,0.0,0.354786,107,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
24329,1427013,28,13,1,2004-04-15,False,NaN,2010-11-02,1994-01-31,12101,NaN,76,0,0,0,0,0,0,0,0,0.0,0.000000,0.0,0.0,0.000000,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [11]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_end', 'data_start', 'matchid',
       'reason_for_removal', 'age_at_index_date', 'insomnia', 'stroke',
       'heart_failure', 'mental_illness_non_smi', 'mental_illness_smi',
       'sleep_apnoea', 'chronic_pulmonary_disease', 'epilepsy',
       'mood_stabilisers_100_pdds', 'benzo_and_z_drugs_100_pdds',
       'other_sedatives_100_pdds', 'antipsychotics_100_pdds',
       'antidepressants_100_pdds', 'non_insomnia_GP_consultations',
       'benzo_and_z_drugs_any', 'insomnia_any', 'insomnia_count:0',
       'insomnia_count:1_5', 'insomnia_count:above_5',
       'non_insomnia_GP_consultations:0', 'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000', 'age_at_index_date:65-69',
       'age_at_index_date:70-74', 'age_at_index_date:75-79',
       'age_at_index_

In [16]:
characteristics = [column for column in pt_features.columns if column not in [
    'patid',  'yob', 'pracid', 'index_date', 'isCase',
    'final dementia medcode', 'data_start', 'data_end', 'matchid']]

In [17]:
baselines = add_baseline_characteristics(characteristics,pt_features)

In [18]:
len(pt_features[pt_features['isCase']==False]),len(pt_features[pt_features['isCase']==True])

(15209, 15209)

In [19]:
baselines[['Cases','Controls']]

,Cases,Controls
female,10055 (66.1%),10055 (66.1%)
stroke,1662 (10.9%),1120 (7.4%)
heart_failure,913 (6.0%),726 (4.8%)
mental_illness_non_smi,5607 (36.9%),3545 (23.3%)
mental_illness_smi,898 (5.9%),528 (3.5%)
sleep_apnoea,25 (0.2%),19 (0.1%)
chronic_pulmonary_disease,3449 (22.7%),2512 (16.5%)
epilepsy,288 (1.9%),165 (1.1%)
benzo_and_z_drugs_any,2311 (15.2%),1684 (11.1%)
insomnia_any,1121 (7.4%),708 (4.7%)
